In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
from bs4 import BeautifulSoup
import requests
import regex as re
from multiprocessing.pool import ThreadPool
from datetime import datetime, timedelta
import random

In [2]:
class DCF:
    def __init__(self, ticker, operatingMargin, RevenueGrowthRate, TaxRate, Costofcapital) -> None:
        self.dfYahoo = yf.Ticker(ticker).history(period="5y", interval="1mo")

In [3]:
from scraper import StockInfo

tick = StockInfo("ORCL")

cash = tick.get_BalanceSheet()

cash

,Breakdown,5/31/2023,5/31/2022,5/31/2021,5/31/2020
0,Total Assets,"134,384,000","109,297,000","131,107,000","115,438,000"
1,Total Liabilities Net Minority Interest,"132,828,000","115,065,000","125,155,000","102,721,000"
2,Total Equity Gross Minority Interest,"1,556,000","-5,768,000","5,952,000","12,717,000"
3,Total Capitalization,"87,976,000","66,342,000","81,947,000","81,943,000"
4,Common Stock Equity,"1,556,000","-5,768,000","5,952,000","12,717,000"
5,Net Tangible Assets,"-70,542,000","-51,019,000","-40,413,000","-34,790,000"
6,Working Capital,"-2,086,000","12,122,000","31,403,000","34,940,000"
7,Invested Capital,"90,481,000","75,859,000","84,245,000","71,597,000"
8,Tangible Book Value,"-70,542,000","-51,019,000","-40,413,000","-34,790,000"
9,Total Debt,"90,481,000","75,859,000","84,245,000","71,597,000"


In [4]:
tick.get_FreeCashFlow()

,Breakdown,ttm,5/31/2023,5/31/2022,5/31/2021,5/31/2020
0,Operating Cash Flow,"17,745,000","17,165,000","9,539,000","15,887,000","13,139,000"
1,Investing Cash Flow,"-8,610,000","-36,484,000","11,220,000","-13,098,000","9,843,000"
2,Financing Cash Flow,"-7,928,000","7,910,000","-29,126,000","-10,378,000","-6,132,000"
3,End Cash Position,"11,655,000","9,765,000","21,383,000","30,098,000","37,239,000"
4,Income Tax Paid Supplemental Data,"3,009,000","2,567,000","3,189,000","3,218,000",NaN
5,Interest Paid Supplemental Data,"3,250,000","2,735,000","2,408,000","1,972,000",NaN
6,Capital Expenditure,"-8,290,000","-8,695,000","-4,511,000","-2,135,000","-1,564,000"
7,Issuance of Capital Stock,"985,000","1,192,000","482,000","1,786,000","1,588,000"
8,Issuance of Debt,"13,948,000","33,994,000",0,"14,934,000","19,888,000"
9,Repayment of Debt,"-16,729,000","-21,050,000","-8,250,000","-2,631,000","-4,500,000"


In [6]:
tick.get_IncomeStatement()

,Breakdown,ttm,5/31/2023,5/31/2022,5/31/2021,5/31/2020
0,Total Revenue,"50,962,000","49,954,000","42,440,000","40,479,000","39,068,000"
1,Cost of Revenue,"14,137,000","13,564,000","8,877,000","7,855,000","7,938,000"
2,Gross Profit,"36,825,000","36,390,000","33,563,000","32,624,000","31,130,000"
3,Operating Expense,"22,503,000","22,617,000","17,733,000","16,842,000","16,928,000"
4,Operating Income,"14,322,000","13,773,000","15,830,000","15,782,000","14,202,000"
5,Net Non Operating Interest Income Expense,"-3,207,000","-3,505,000","-2,755,000","-2,496,000","-1,995,000"
6,Other Income Expense,"-1,270,000","-1,142,000","-5,426,000","-287,000","-144,000"
7,Pretax Income,"9,845,000","9,126,000","7,649,000","12,999,000","12,063,000"
8,Tax Provision,"470,000","623,000","932,000","-747,000","1,928,000"
9,Net Income Common Stockholders,"9,375,000","8,503,000","6,717,000","13,746,000","10,135,000"
